In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import PIL 
image = PIL.Image.open('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/IM-0219-0001.jpeg')

In [3]:
image.size

In [4]:
# Your code here; transform the image files and then load them into Keras as tensors 
# (be sure to perform a train-val-test split)
import keras

# Instantiating a generator object and normalizing the RGB values
traingen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
testgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
valgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_data = traingen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/train',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    seed=42
)

test_data = testgen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/test',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    seed=42
)

val_data = valgen.flow_from_directory(
    directory='../input/chest-xray-pneumonia/chest_xray/val',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20,
    seed=42
)

In [5]:
input_shape = (150,150,3)
output_shape = 2

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

# Model building in a single list
baseline = keras.Sequential(
    [
        keras.Input(shape=input_shape), # Don't always need this input separately
        layers.Flatten(), # need to flatten our images to be one long array
        layers.Dense(64, activation="tanh"),
        layers.Dense(1, activation="sigmoid"), #sigmoid for binary
    ])

baseline.summary()

In [7]:
batch_size = 20
epochs = 10

# Compiling our model
baseline.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["acc"])
#loss different for binary

In [8]:
baseline_results = baseline.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data)

In [9]:
baseline_results.history

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Defining a results visualization function
def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

In [11]:
score = baseline.evaluate(test_data, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

visualize_training_results(baseline_results)